In [ ]:
!pip install --upgrade git+https://github.com/EmGarr/od.git
# Useful for tensorboard
!pip install --upgrade grpcio

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Download COCO/2017

Download and preprocess COCO/2017 to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }
```

If you need to download the dataset in a specific directory you can use the argument `data_dir` of `tfds.load`.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from od.dataset.preprocessing import preprocess_coco_example, expand_dims_for_single_batch
from od.core.standard_fields import BoxField

ds_train, ds_info = tfds.load(name="coco/2017", split="train", shuffle_files=True, with_info=True)
ds_train = ds_train.map(preprocess_coco_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
ds_train =  ds_train.filter(lambda x: tf.shape(x[BoxField.BOXES])[0] > 1)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = tfds.load(name="coco/2017", split="validation", shuffle_files=False)
ds_test = ds_test.map(preprocess_coco_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
ds_test =  ds_test.filter(lambda x: tf.shape(x[BoxField.BOXES])[0] > 1)
ds_test = ds_test.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_info

# Load and train the network


In [ ]:
from od.model.faster_rcnn import build_fpn_resnet50_faster_rcnn
from od.core.standard_fields import BoxField
from od.core.learning_rate_schedule import WarmupLearningRateScheduler
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Number of classes of COCO
classes = ds_info.features['objects']['label'].names
num_classes = len(classes)
batch_size = 1

model_faster_rcnn = build_fpn_resnet50_faster_rcnn(num_classes, batch_size)
base_lr = 0.005
optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr)
model_faster_rcnn.compile(optimizer=optimizer, loss=None)
callbacks = [WarmupLearningRateScheduler(base_lr, 1, epochs=[8, 10], init_lr=0.0001), TensorBoard(), ModelCheckpoint('.checkpoints/')]

model_faster_rcnn.fit(ds_train, validation_data=ds_test, epochs=11, callbacks=callbacks)

In [ ]:
# Save the weights for the serving
model_faster_rcnn.save_weights('final_weights.h5')

# Tensorboard

In [ ]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs

# Export for inference

When you are in training mode all the ground_truths are used as inputs:
- BoxField.BOXES
- BoxField.LABELS
- BoxField.NUM_BOXES
- BoxField.WEIGHTS

We want to remove those for the serving.

The first step is to rebuild the graph in inference mode. Reload the `final_weights.h5` and save using the save method from `tf.keras.Model`.


In [ ]:
from od.model.faster_rcnn import build_fpn_resnet50_faster_rcnn

model_faster_rcnn_inference = build_fpn_resnet50_faster_rcnn(num_classes, None, training=False)
model_faster_rcnn_inference.load_weights('final_weights.h5')
model_faster_rcnn_inference.save('serving_model', include_optimizer=False)

## Coco evaluation


### Load the dataset

In [ ]:
import tensorflow_datasets as tfds

ds_val, ds_info = tfds.load(name="coco/2017", split="validation", shuffle_files=False, with_info=True)
# category_ids basicaly map the index 0 the id
# e.g: 0 -> 1, 2 -> 3, 79 -> 90
category_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]

### Super dirty but the evaluation works

1. perform the analysis

In [ ]:
import json
import numpy as np
import tensorflow as tf

from od.core.standard_fields import DatasetField, BoxField             
from od.core.box_ops import convert_to_center_coordinates              
from od.dataset.preprocessing import resize_to_min_dim                 
                                                                       
results = []                                                           
                                                                       
for i, example in enumerate(ds_val): 
    print(i)
    # preprocess image                                                 
    image = resize_to_min_dim(example['image'], 800.0, 1300.0)         
    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32) 
    inputs = {                                                         
        DatasetField.IMAGES: tf.expand_dims(image, axis=0),            
        DatasetField.IMAGES_INFO: tf.expand_dims(image_information, axis=0)
    }                                                                  
    # predict                                                          
    nmsed_boxes, nmsed_scores, nmsed_labels, valid_detections = model_faster_rcnn_inference.predict(inputs)
                                                                       
    # Post processing and append to coco results                       
    bbox = nmsed_boxes[0] * tf.tile(tf.expand_dims(tf.cast(example['image'].shape[:2], tf.float32)                    
        , axis=0), [1, 2])                   
    scores = nmsed_scores[0]                                           
    labels = nmsed_labels[0]                                           
    for i in range(valid_detections[0]):
        # Convert from [y_min, x_min, y_max, x_max] to coco format [x_min, y_min, w, h]
        sbox = bbox[i].numpy()
        sbox = [sbox[1], sbox[0], sbox[3] - sbox[1], sbox[2] - sbox[0]]
        res = {                                                        
                'image_id': int(example['image/id']),                       
                'category_id': category_ids[int(labels[i])],           
                'bbox': [round(float(c), 4) for c in sbox],
                'score': round(float(scores[i]), 4),                     
            }                                                          
        results.append(res)                                            
                                                   
                                                                       
with open('coco_results.json', 'w') as f:                              
    json.dump(results, f)

2. install the coco library to compute the performances

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open('coco_results_corrected.json', 'r') as f:                              
    results = json.load(f)
coco = COCO('./annotations/instances_val2017.json')
ret = {}

cocoDt = coco.loadRes(results)
cocoEval = COCOeval(coco, cocoDt, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()